In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# A one dimensional Ornstein−Uhlenbeck process

## Definition

A simple Ornstein--Uhlenbeck process takes the form
$$\mathrm{d}y(t) = -\theta y(t)\mathrm{d}t + \sigma \mathrm{d}W(t), $$
with $\theta$ the drift term and $\sigma$ diffusion term. The term $\mathrm{d}W(t)$ comprises the Brownian motion

## Integrating with an Euler−Maruyama scheme

This is the equivalent of the simplest Euler integrator for stochastic processes. You can find more specifics on the Euler−Maruyama onwikipedia

In [ ]:
# The drift and diffusion
theta = 0.3
sigma = 0.1

# The total integration time
t_start = 0
t_final = 500

# The desired timestep of integration
delta_t = 0.001

# time array of the process
time = np.linspace(0, t_final, t_final * int(1 / delta_t))

For the integration initialise an array $y$ and a normally distributed noise $\mathrm{d}W(t)$

In [ ]:
# Initialise the array y
y = np.zeros([time.size])

# Generate a Wiener process with a scale of np.sqrt(delta_t)
dW = np.random.normal(loc = 0, scale = np.sqrt(delta_t), size = [time.size,1])
# you can equivalently use np.sqrt(delta_t) * np.random.normal(loc=0, scale=1,size=[time.size,1])

# Give some small random initial conditions
y[0]=np.random.normal(size = 1) / 10

# Integrate the process
for i in range(1, time.size):
    y[i] = y[i-1] - theta * y[i-1] * delta_t + sigma * dW[i] 

## Visualising the process

In [ ]:
#This is the stochastic trajectory over time
plt.plot(time, y , label = r'Trajectory of O–U process')

plt.xlabel(r'time $t$')
plt.ylabel(r'$y(t)$')
plt.legend()

## Obtaining the Kramers−Moyal coefficients

We will now implement the KM package to calculate the Kramers−Moyal coefficients

In [ ]:
# Import your (favourite) package to calculate Kramers−Moyal coefficients
# You will need only km to perform the calculation
from kramersmoyal import km

In [ ]:
# Choose the size of your target space 
bins = np.array([5000])

# Introduce the desired orders to calculate
# Please keep the  [0]  term. It is the normalisation. 
powers = np.array([[0], [1], [2], [3], [4], [5], [6], [7], [8]])

# Choose a desired bandwidth bw
# this is similar to the inverse of number of bins
bw = 0.1

# Calculate the Kramers−Moyal coefficients
kmc, edges = km(y, bw = bw, bins = bins, powers = powers)

# The K−M coefficients are stacked along the fisrt dim of the
# kmc array, so kmc[1,:] is the first K−M coefficient, kmc[2,:]
# is the second, etc. etc.

## Visualising the Kramers−Moyal coefficients

Lets now visualise the obtained Kramers−Moyal coefficients
Firstly, let's plot the first Kramers−Moyal coefficients, i.e., the drift coefficient

In [ ]:
# Lets restrict the plot to where we have good statistics
plt.plot(edges[0][1500:-1500], kmc[1,1500:-1500], label = r'First K−M coefficient: drift')

# And to guide the eye, here is the actual theoretical value,
# which is an inclined line, with slope -theta*delta_t
plt.plot(edges[0][1500:-1500], -edges[0][1500:-1500] * theta * delta_t, ':', label = r'Theory', color=r'black')

plt.xlabel(r'$y(t)$')
plt.ylabel(r'K−M coefficient')
plt.legend()

And now the second Kramers−Moyal coefficients, i.e., the diffusion coefficient

In [ ]:
# Lets restrict the plot to where we have good statistics
plt.plot(edges[0][1500:-1500], kmc[2,1500:-1500], label = r'Second K−M coefficient: diffusion')

# And to guide the eye, here is the actual theoretical value,
# which is a constant of value sigma**2 * delta_t /2
plt.plot(edges[0][1500:-1500], np.ones(edges[0][1500:-1500].size) * sigma**2 * delta_t / 2, ':', label = r'Theory', color=r'black')

plt.xlabel(r'$y(t)$')
plt.ylabel(r'K−M coefficient')
plt.ylim([sigma**2 * delta_t / 4, sigma**2 * delta_t])
plt.legend()

# A bivariate (two-dimensional) Jump-Diffusion process

A more involved example is a bivariate Jump-Diffusion process, that takes the form
$$\begin{pmatrix}
    \mathrm{d}y_1(t) \\ \mathrm{d}y_2(t)
    \end{pmatrix}=
    \begin{pmatrix}
    N_1 \\ N_2
    \end{pmatrix}
    \mathrm{d} t +
    \begin{pmatrix}
    g_{1,1} & g_{1,2} \\
    g_{2,1} & g_{2,2}
    \end{pmatrix}
    \begin{pmatrix}
    \mathrm{d}W_1 \\ \mathrm{d}W_2
    \end{pmatrix}+
    \begin{pmatrix}
    \xi_{1,1} & \xi_{1,2} \\
    \xi_{2,1} & \xi_{2,2}
    \end{pmatrix}
    \begin{pmatrix}
    \mathrm{d}J_1 \\ \mathrm{d}J_2
    \end{pmatrix}
$$
with $(N_1, N_2)$ the drift vector, $g$ the diffusion matrix, and $\xi$ the jump matrix. The term $(\mathrm{d}W_1(t),\mathrm{d}W_2(t))$ comprises the Brownian motions and $(\mathrm{d}J_1, \mathrm{d}J_2)$ comprises the Poisson jumps.

## Integrating with an Euler−Maruyama scheme

In [ ]:
# Define first the drift vector N with some exemplary values
N = np.array([1.0, 0.5])

# Define the diffusion matrix g with some exemplary values
g = np.array([[0.5, 0.2],[0.1, 1.0]])

# and the jump amplitude matrix xi
xi = np.array([[0.2, 0.5],[0.4, 0.5]])

# lastly, the jump frequency lambda (l)
l = np.array([0.1, 0.2])

# The total integration time
t_start = 0
t_final = 500

# The desired timestep of integration
delta_t = 0.001

# time array of the process
time = np.linspace(0, t_final, t_final * int(1 / delta_t))

Initialise an (n,2)-array $y$, (n,2)-normally distributed noise $\mathrm{d}W_{1,2}(t)$, and (n,2)-Poisson processes $\mathrm{d}J_{1,2}(t)$

In [ ]:
# Initialise the array y
y = np.zeros([time.size, 2])

# Generate two Wiener processes with a scale of np.sqrt(delta_t)
dW = np.random.normal(loc = 0, scale = np.sqrt(delta_t), size = [time.size, 2])

# Generate two Poisson process with a delta_t scale
dJ = np.zeros([ time.size, 2 ])
dJ[:,0] = np.random.poisson(lam = l[0] * delta_t, size = time.size)
dJ[:,1] = np.random.poisson(lam = l[1] * delta_t, size = time.size)

# Give some small random initial conditions
y[0,:]=np.random.normal(size = 2) / 10

# Integrate the process, with a two step process
for i in range(1, time.size):
    y[i,0] = y[i-1,0]  -  N[0] * y[i-1,0] * delta_t  +  g[0,0] * dW[i,0] +  g[0,1] * dW[i,1]
    y[i,1] = y[i-1,1]  -  N[1] * y[i-1,1] * delta_t  +  g[1,0] * dW[i,0] +  g[1,1] * dW[i,1]
    
    if dJ[i,0]>0 or dJ[i,1]:
        # Generate here the normally distributed matrix xi of the jumps
        s_00 = np.random.normal(loc = 0, scale = np.sqrt(xi[0,0]))
        s_01 = np.random.normal(loc = 0, scale = np.sqrt(xi[0,1]))
        s_10 = np.random.normal(loc = 0, scale = np.sqrt(xi[1,0]))
        s_11 = np.random.normal(loc = 0, scale = np.sqrt(xi[1,1]))
        
        y[i,0] += s_00 * dJ[i,0]  +  s_01 * dJ[i,1]
        y[i,1] += s_10 * dJ[i,0]  +  s_11 * dJ[i,1]

Still in progress, now the Kramers−Moyal coefficients in two dimensions